## [ 한글 데이터 셋 RNN ]<hr>
- 데이터셋 : Korpora의 nsmc

[1] 데이터 준비 <hr>

In [1]:
from Korpora import Korpora
import pandas as pd

In [2]:
# 데이터 저장 및 확인
corpus = Korpora.load('nsmc')
corpusDF = pd.DataFrame(corpus.test)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\kdp\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\kdp\K

In [3]:
# 학습용, 테스트용 데이터 분리
trainDF = corpusDF.sample(frac = 0.9, random_state=42)
testDF = corpusDF.drop(trainDF.index)

In [4]:
trainDF, testDF

(                                                    text  label
 33553  모든 편견을 날려 버리는 가슴 따뜻한 영화. 로버트 드 니로, 필립 세이모어 호프만...      1
 9427                    무한 리메이크의 소재. 감독의 역량은 항상 그 자리에...      0
 199                                          신날 것 없는 애니.      0
 12447                                              잔잔 격동      1
 39489                                 오랜만에 찾은 주말의 명화의 보석      1
 ...                                                  ...    ...
 22164                                    진짜 후하게 준 거 알지??      0
 44594                                   한국 저예산+공포 영화의 극치      0
 4996            그지같은영화 난해하고 뭘얘기하는건지 도중에 보다가 나왔었던적은 첨인 영화      0
 3262                                                  10      1
 6838                   이것 또한 사랑이다 회전 그네씬 계속 돌려보는데 눈물이나네요      1
 
 [45000 rows x 2 columns],
                                                     text  label
 9         이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..      1
 55                                   난 재밌던데 평점 왜케 낮지 ``;    

In [5]:
# !pip install tabulate

In [6]:
print(trainDF.head(5).to_markdown())
print('train size', len(trainDF))
print('test size', len(testDF))

|       | text                                                                                     |   label |
|------:|:-----------------------------------------------------------------------------------------|--------:|
| 33553 | 모든 편견을 날려 버리는 가슴 따뜻한 영화. 로버트 드 니로, 필립 세이모어 호프만 영원하라. |       1 |
|  9427 | 무한 리메이크의 소재. 감독의 역량은 항상 그 자리에...                                    |       0 |
|   199 | 신날 것 없는 애니.                                                                       |       0 |
| 12447 | 잔잔 격동                                                                                |       1 |
| 39489 | 오랜만에 찾은 주말의 명화의 보석                                                         |       1 |
train size 45000
test size 5000


[2] 단어 사전 생성 <hr>
- 토큰화 진행
- 단어 사전 생성

[2-1] 토큰화 진행 : 문장 -> 단어

In [7]:
from konlpy.tag import Okt
from collections import Counter

# 토큰화 인스턴스 생성
tokenizer = Okt()

In [8]:
# 문장 ==> 단어 분리
for text in trainDF.text:
    print(text)
    break

모든 편견을 날려 버리는 가슴 따뜻한 영화. 로버트 드 니로, 필립 세이모어 호프만 영원하라.


In [9]:
trainDF.text.values

array(['모든 편견을 날려 버리는 가슴 따뜻한 영화. 로버트 드 니로, 필립 세이모어 호프만 영원하라.',
       '무한 리메이크의 소재. 감독의 역량은 항상 그 자리에...', '신날 것 없는 애니.', ...,
       '그지같은영화 난해하고 뭘얘기하는건지 도중에 보다가 나왔었던적은 첨인 영화', '10',
       '이것 또한 사랑이다 회전 그네씬 계속 돌려보는데 눈물이나네요'], dtype=object)

In [10]:
# train = p.S_preprocess(trainDF.text.values)
# test = p.S_preprocess(testDF.text.values)

In [11]:
# 토큰화해서 리스트안에 저장
train_tokens = [tokenizer.morphs(str(txt), stem=True) for txt in trainDF.values]
test_tokens = [tokenizer.morphs(str(txt), stem=True) for txt in testDF.values]

In [12]:
print(f'[train_tokens]     {len(train_tokens)}개')
print(f'[test_tokens]      {len(test_tokens)}개')
print(f'[train_tokens[0]]  {len(train_tokens[0])}개')
print(f'[test_tokens[0]]   {len(test_tokens[0])}개') 

[train_tokens]     45000개
[test_tokens]      5000개
[train_tokens[0]]  22개
[test_tokens[0]]   21개


[2-2] 토큰화 -> 단어/어휘 사전 생성

In [13]:
# 단어 사전 생성 함수
# 단어사전 생성하기
def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter() # 단어 세는 애

    # 단어 / 토큰에 대한 빈도 수 계산
    for tokens in corpus:
        counter.update(tokens) # dict니까

    # 단어 / 어휘 사전 생성하기
    vocab = special_tokens

    # 빈도수가 높은 단어부터 단어사전에 추가하기. 
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab

In [14]:
# 학습용 데이터셋 단어/어휘 사전 생성

# - n_vocab : 단어사전 크기, 가장 많이 등장한 상위 n_vocab개의 단어만 사용
# - special_tokens : 특수 토큰(Padding, Unknown) 추가
vocab = build_vocab(train_tokens, n_vocab=5000, special_tokens=['<PAD>', '<UNK>']) # 0, 1 번째 인덱스가 추가된 채로 단어 사전 생성

[2-3] 인코딩 & 디코딩 인덱싱

In [15]:
### 인코딩 : 문자 > 숫자 변환
token2id = {v:idx for idx, v in enumerate(vocab)}

### 인코딩 : 숫자 > 문자 변환
id2token = {idx:v for idx, v in enumerate(vocab)}

In [16]:
# 리뷰에 문자를 정수로 변환 및 단어/어휘 사전에 없는 문자도 처리
UNK_ID =token2id.get('<UNK>')  # 0

# 단어가 사전에 없으면 0을 반환해서 리스트 내에 넣으라는 뜻
trainID = [[token2id.get(token, UNK_ID) for token in text] for text in train_tokens]
testID = [[token2id.get(token, UNK_ID) for token in text] for text in test_tokens]

[3] 데이터 가공
- 데이터 길이 표준화 => 다른 길이의 데이터 길이 맞추기 -> 1개 문장 구성하는 단어의 개수 맞추기

[3-1] 토큰 정수화

[3-2] 데이터 구성 단어 수 맞추기 즉, 패딩(padding)
-단어 수 선정 필요
- 선정된 단어 수에 맞게 데이터 조절 => 길면 잘라내기, 짧으면 채우기

In [17]:
import numpy as np

In [18]:
# 패딩 처리 함수
# sentences  : 토큰화된 문장 데이터
# max_length : 최대 문장 길이, 즉 1개 문장 구성 단어 수
# pad : 패딩 처리시 추가될 문자 값
# start : 패딩시 처리 방향 [기본값] r : 오른쪽 즉, 뒷부분 자르기/추가하기
def pad_sequences(sentences, max_length, pad, start = 'R'):
    result = []
    for sen in sentences:
        sen = sen[:max_length] if start == 'R' else sen[:-max_length] #start 매개변수가 R이면 오른쪽에서 잘라내기
        padd_sen = sen + [pad]*(max_length - len(sen)) if start =='R' else ([pad]*(max_length - len(sen)) + sen) # start 매개변수가 R이면 오른쪽부터 패딩 넣기
        result.append(padd_sen)

    return result

In [19]:
# 학습용, 테스트용 데이터 패딩 처리
PAD_ID = token2id.get('<PAD>')
MAX_LENGTH = 32  # 학습할 때 총 32개의 히든 스테이트를 갖게 됨

# 문장 당 32개의 단어들만 존재
train_ids = pad_sequences(trainID, MAX_LENGTH, PAD_ID)
test_ids = pad_sequences(testID, MAX_LENGTH, PAD_ID)

In [20]:
# 0번째 문장을 꺼내봤더니 단어갯수가 32개 => 잘 패딩된 것을 확인할 수 있다.
print(f'[train_ids] ---> {len(train_ids[0])}개')
print(f'[test_ids] ---> {len(test_ids[0])}개')

[train_ids] ---> 32개
[test_ids] ---> 32개


[4] 학습 준비
* 데이터 로더 준비
* 학습용/테스트용 함수
* 모델 클래스
* 학습 관련 변수 => DEVICE, EPOCH, BATCH_SIZE,  LOSS_FN, MODEL 등

[4-1]  데이터 로더 준비

In [21]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn

In [22]:
len(trainID), len(trainDF.label)

(45000, 45000)

In [23]:
## 데이터셋 생성 : list -> tensor
# 학습용 데이터셋
dataTS = torch.LongTensor(train_ids)
labelTS = torch.tensor(trainDF.label.values, dtype=torch.float32)


print(dataTS.shape, labelTS.shape)
trainDS = TensorDataset(dataTS, labelTS)


# 테스트용 데이터셋
testdataTS = torch.LongTensor(test_ids)
testlabelTS = torch.tensor(testDF.label.values, dtype=torch.float32)
testDS = TensorDataset(testdataTS, testlabelTS)


torch.Size([45000, 32]) torch.Size([45000])


In [24]:
# 데이터 로더 생성
BATCH_SIZE = 32

trainDL = DataLoader(trainDS, batch_size = BATCH_SIZE, shuffle = True)
testDL = DataLoader(testDS, batch_size = BATCH_SIZE, shuffle = True)

[4-2] 모델 클래스 정의
* 입력층 : Embedding Layer
* 은닉층 : RNN / LSTM Layer
* 은닉층 : Dropout Layer
* 출력층 : Linear Layer

In [25]:
class SentenceClassifier(nn.Module):
    def __init__(
            self,
            n_vocab,
            hidden_dim,
            embedding_dim,
            n_layers,
            dropout=0.5,
            bidirectional=True,
            model_type='lstm'
    ):
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=embedding_dim,
            padding_idx=0
        )
        if model_type == 'rnn':
            self.model = nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        elif model_type =='lstm':
            self.model = nn.LSTM(
                input_size = embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        if bidirectional:
            self.classifier = nn.Linear(hidden_dim*2, 1)
        else:
            self.classifier = nn.Linear(hidden_dim,1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:,-1,:]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [33]:
# 학습 관련 변수
MODEL = SentenceClassifier(len(token2id),64,128,2)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 5
LOSS_FN = nn.BCEWithLogitsLoss()
OPTIMIZER = torch.optim.Adam(MODEL.parameters(), lr=0.001)

In [34]:
len(trainDL.dataset)

45000

In [35]:
def training(model, DL, loss_fn, optimizer):
    model.train()
    running_loss = 0.0
    for ids, label in DL:

        label = label.to(DEVICE).unsqueeze(1)
        ids = ids.to(DEVICE)

        output = model(ids)
        
        loss = loss_fn(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    train_loss = running_loss / len(DL.dataset)
    return train_loss

In [36]:
def testing(model, DL, loss_fn):
    model.eval()
    losses = []
    correct = []
    with torch.no_grad():
        for ids, label in DL:
            label = label.to(DEVICE).unsqueeze(1)
            ids = ids.to(DEVICE)

            output = model(ids)
            loss = loss_fn(output, label) 
            losses.append(loss.item())
            yhat = torch.sigmoid(output) > 0.5
            correct.extend(
                torch.eq(yhat, label).cpu().tolist()
            )
        
    val_loss = np.mean(losses)
    val_acc = np.mean(correct) 
    return val_loss, val_acc

In [37]:
import numpy as np

train_losses = []
test_losses = []
test_acc = []

for epoch in range(EPOCHS):
    train_loss = training(MODEL, trainDL, LOSS_FN, OPTIMIZER)
    val_loss, val_acc = testing(MODEL, testDL, LOSS_FN)
    
    train_losses.append(train_loss)
    test_losses.append(val_loss) 
    test_acc.append(val_acc)

    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss : {train_loss:.4f}, Val Loss : {val_loss:.4f}, Val Acc : {val_acc:.4f}")

Epoch 1/5, Train Loss : 0.0032, Val Loss : 0.0904, Val Acc : 0.9676
Epoch 2/5, Train Loss : 0.0016, Val Loss : 0.0580, Val Acc : 0.9708
Epoch 3/5, Train Loss : 0.0011, Val Loss : 0.0578, Val Acc : 0.9752
Epoch 4/5, Train Loss : 0.0008, Val Loss : 0.0740, Val Acc : 0.9732
Epoch 5/5, Train Loss : 0.0006, Val Loss : 0.0786, Val Acc : 0.9744


In [ ]:
# 모델 평가
train_loss_avg = np.mean(train_losses)
test_loss_avg = np.mean(test_losses)
print(f"Train Loss : {train_loss_avg:.4f}  Test Loss: {test_loss_avg:.4f}")